# This project implemented by Elham Mahdipour
## She is a Ph.D. Candidate of computer engineering at Yazd University, Yazd, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ elham.mahdipour@stu.yazd.ac.ir

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_leda('biogrid samples\CElegans.gw')
G1

In [2]:
G2=nx.read_leda('biogrid samples\AThaliana.gw')
G2

In [3]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

3134
5897
*******
3134
5897


In [4]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\CE-AT.tsv')
G_target_na  

In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
elt=G_target_na.edges()
elt=list(elt)
elt

[('176191', '833987'),
 ('173074', '829969'),
 ('183226', '832673'),
 ('172249', '820408'),
 ('3564970', '844228'),
 ('173414', '832096'),
 ('180324', '819375'),
 ('177325', '838425'),
 ('172611', '829427'),
 ('171696', '834747'),
 ('178259', '832836'),
 ('177679', '825469'),
 ('177845', '817622'),
 ('183942', '827501'),
 ('3565580', '816345'),
 ('172499', '825755'),
 ('172589', '814698'),
 ('178960', '837733'),
 ('6418789', '841539'),
 ('177497', '819717'),
 ('171821', '817673'),
 ('172984', '826665'),
 ('175070', '820266'),
 ('175669', '821628'),
 ('176559', '817259'),
 ('177200', '837734'),
 ('177120', '830044'),
 ('172232', '839636'),
 ('176793', '820385'),
 ('178185', '820218'),
 ('175512', '820293'),
 ('172432', '815823'),
 ('176291', '817515'),
 ('179459', '817336'),
 ('176841', '831071'),
 ('179091', '821689'),
 ('176321', '836220'),
 ('177403', '839521'),
 ('13216595', '824778'),
 ('181999', '826596'),
 ('179562', '815080'),
 ('178017', '823849'),
 ('172864', '844406'),
 ('176

In [13]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [14]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [15]:
import numpy as np
m, coeff_pr, dd, cd, seq=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [16]:
# en_mat is encoding matrix
en_mat=[]

for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
        else:
            align_class='No'
        
        sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
        en_mat.append(sample)
print(len(en_mat))

18481198


In [17]:
yc=[]
noc=[]
for i in range(len(en_mat)):
    if en_mat[i][7]=='Yes': 
        yc.append(en_mat[i])
    else:
        noc.append(en_mat[i])
print(len(yc), len(noc))

129 18481069


In [18]:
noc.sort(key = lambda i: i[6])
data=yc+noc[0:len(yc)] 
len(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

(258, 258)

In [19]:
X_train=X
y_train=y

# test model for real data

In [21]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [22]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: LSTM-NA network

## Load tune LSTM-NA model

In [23]:
from keras.models import load_model
#del model_onelstm
model_onelstm = load_model('biogrid samples/deep_model_undersample_lstm_SC-AT.h5')
model_onelstm.load_weights('biogrid samples/deep_model_undersample_lstm_weights_SC-AT.h5', by_name=True)

In [25]:
# without resample
result_tr = model_onelstm.evaluate(X_train, y_train)
result_tr

258/258 [==============================] - 0s 274us/step


[0.00208963016806127, 1.0, 0.0020873358007520437, 4.613730197888799e-06]

In [24]:
%%time
out_tr=model_onelstm.predict(X_train)
out_tr

Wall time: 244 ms


array([[0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.00158065, 0.9984194 ],
       [0.

In [26]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

[[129   0]
 [  0 129]]


In [27]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

1.0
1.0
1.0


# test RNN with tuned resample model

In [28]:
from keras.models import load_model
#del model
model_rnn = load_model('biogrid samples/deep_model_undersample_RNN_SC-AT.h5')
model_rnn.load_weights('biogrid samples/deep_model_undersample_RNN_weights_SC-AT.h5', by_name=True)

In [29]:
# without resample
result_tr = model_rnn.evaluate(X_train, y_train)
result_tr

258/258 [==============================] - 0s 259us/step


[0.0011415502007747459, 1.0, 0.0011408942518755794, 1.3051846963207936e-06]

In [30]:
%%time
out_tr=model_rnn.predict(X_train)
out_tr

Wall time: 73.8 ms


array([[0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.00108135, 0.99891865],
       [0.

In [31]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

[[129   0]
 [  0 129]]


In [32]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

1.0
1.0
1.0
